# CrewAI basic Multi-Agent LLM App: multi-agent app to replace the content marketing team

In [1]:
#!pip install python-dotenv

In [18]:
pip install crewai==0.28.8


Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic Multiagent App with CrewAI

In [4]:
#!pip install crewai beautifulsoup4

* We will call our LangSmith project "basiccrewai".
* Create a .gitignore file and write .env there to keep the .env file confidential.

## Add the Tavily API key in your .env file

To get a Tavily API key, you can follow these simple steps:

1. **Sign Up or Log In:** 
   - Visit the Tavily website.
   - If you don't have an account, sign up by providing your details. If you already have an account, just log in.

2. **Navigate to API Section:**
   - Once logged in, find the section or menu item that mentions "API" or "Developer Tools."
   - This is usually in your account settings or a dedicated "API" page.

3. **Create a New API Key:**
   - Look for an option like "Generate API Key" or "Create New Key."
   - Click on it, and the system will generate an API key for you.

4. **Copy the API Key:**
   - Once the key is generated, it will be displayed on the screen.
   - Copy it carefully and store it securely. You’ll need this key to access Tavily's API.

5. **Use the API Key:**
   - Use this key in your application or service to authenticate and interact with Tavily's API.

That's it! You now have your Tavily API key and can start using it for your projects.

* Add your Tavily API key in your .env file:

TAVILY_API_KEY=yourkey

## Import the necessary modules

If you are using the pre-loaded poetry shell, you do not need to install the following packages because they are already pre-loaded for you:

In [5]:
#!pip install python-dotenv typing

## Initial imports
From now on we will comment the code out so it will not be executed in the notebook. In the following steps you will learn how to execute the code in the code editor.

In [21]:
import requests
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv()

True

## Select the LLM

In [20]:
llm = ChatOpenAI(model="gpt-4-turbo-preview") 

## Define the tools to use
* Online searching with Tavily
* HTML parsing with BeautifulSoup
* return_direct=False
    * Meaning: this tool will be returning results privately to the agent, not publicly to the app.

In [8]:
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Used to process content found on the internet."""
    response = requests.get(url=url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()

tools = [TavilySearchResults(max_results=1), process_search_tool]


* Important note: for demo and cost purposes, we have set the max_results of the online search with Tavily to only 1. This indeed will limit the quality of the online search since our online search agent will only study one search result instead of comparing several results.
* When you work in a professional app, set the max_results to a higher number but be careful with the const implications.

## Create the online_researcher agent

In [ ]:
online_researcher = Agent(
    role=" Researcher",Online
    goal="Research the topic online",
    backstory="""Your primary role is to function as an intelligent online research assistant, adept at scouring 
    the internet for the latest and most relevant trending stories across various sectors like politics, technology, 
    health, culture, and global events. You possess the capability to access a wide range of online news sources, 
    blogs, and social media platforms to gather real-time information.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

## Create the blog_manager agent

In [10]:
blog_manager = Agent(
    role="Blog Manager",
    goal="Write the blog article",
    backstory="""You are a Blog Manager. The role of a Blog Manager encompasses several critical responsibilities aimed at transforming initial drafts into polished, SEO-optimized blog articles that engage and grow an audience. Starting with drafts provided by online researchers, the Blog Manager must thoroughly understand the content, ensuring it aligns with the blog's tone, target audience, and thematic goals. Key responsibilities include:

1. Content Enhancement: Elevate the draft's quality by improving clarity, flow, and engagement. This involves refining the narrative, adding compelling headers, and ensuring the article is reader-friendly and informative.

2. SEO Optimization: Implement best practices for search engine optimization. This includes keyword research and integration, optimizing meta descriptions, and ensuring URL structures and heading tags enhance visibility in search engine results.

3. Compliance and Best Practices: Ensure the content adheres to legal and ethical standards, including copyright laws and truth in advertising. The Blog Manager must also keep up with evolving SEO strategies and blogging trends to maintain and enhance content effectiveness.

4. Editorial Oversight: Work closely with writers and contributors to maintain a consistent voice and quality across all blog posts. This may also involve managing a content calendar, scheduling posts for optimal engagement, and coordinating with marketing teams to support promotional activities.

5. Analytics and Feedback Integration: Regularly review performance metrics to understand audience engagement and preferences. Use this data to refine future content and optimize overall blog strategy.

In summary, the Blog Manager plays a pivotal role in bridging initial research and the final publication by enhancing content quality, ensuring SEO compatibility, and aligning with the strategic objectives of the blog. This position requires a blend of creative, technical, and analytical skills to successfully manage and grow the blog's presence online.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

## Create the social_media_manager agent

In [11]:
social_media_manager = Agent(
    role="Social Media Manager",
    goal="Write a tweet",
    backstory="""You are a Social Media Manager. The role of a Social Media Manager, particularly for managing Twitter content, involves transforming research drafts into concise, engaging tweets that resonate with the audience and adhere to platform best practices. Upon receiving a draft from an online researcher, the Social Media Manager is tasked with several critical functions:

1. Content Condensation: Distill the core message of the draft into a tweet, which typically allows for only 280 characters. This requires a sharp focus on brevity while maintaining the essence and impact of the message.

2. Engagement Optimization: Craft tweets to maximize engagement. This includes the strategic use of compelling language, relevant hashtags, and timely topics that resonate with the target audience.

3. Compliance and Best Practices: Ensure that the tweets follow Twitter’s guidelines and best practices, including the appropriate use of mentions, hashtags, and links. Also, adhere to ethical standards, avoiding misinformation and respecting copyright norms.

In summary, the Social Media Manager's role is crucial in leveraging Twitter to disseminate information effectively, engage with followers, and build the brand’s presence online. This position combines creative communication skills with strategic planning and analysis to optimize social media impact.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

## Create the content_marketing_manager agent
* Pay attention to the lines "verbose=False" and "allow delegation=True"

In [12]:
content_marketing_manager = Agent(
    role="Content Marketing Manager",
    goal="Manage the Content Marketing Team",
    backstory="""You are an excellent Content Marketing Manager. Your primary role is to supervise each publication from the 'blog manager' 
    and the tweets written by the 'social media manager' and approve the work for publication. Examine the work and regulate violent language, abusive content and racist content.
    
    Capabilities:

    Editorial Review: Analyze the final drafts from the blog manager and the social media manager for style consistency, thematic alignment, and overall narrative flow.

    Quality Assurance: Conduct detailed checks for grammatical accuracy, factual correctness, and adherence to journalistic standards in the news content, as well as creativity and effectiveness in the advertisements.

    Feedback Loop: Provide constructive feedback to both the blog manager and social media manager, facilitating a collaborative environment for continuous improvement in content creation and presentation.""",
    verbose=True,
    allow_delegation=True,
    tools=tools,
    llm=llm
)

## Define the tasks

In [13]:
task1 = Task(
    description="""Write me a report on Agentic Behavior. After the research on Agentic Behavior,pass the 
    findings to the blog manager to generate the final blog article. Once done, pass it to the social media 
    manager to write a tweet on the subject.""",
    expected_output="Report on Agentic Behavior",
    agent=online_researcher
)

In [14]:
task2 = Task(
    description="""Using the research findings of the news correspondent, write an article for the blog. 
    The publication should contain links to sources stated by the online researcher. 
    Your final answer MUST be the full blog post of at least 3 paragraphs.""",
    expected_output="Blog Article",    
    agent=blog_manager
)

In [15]:
task3 = Task(
    description="""Using the research findings of the news correspondent, write a tweet. Your final answer MUST be 
    the full tweet.""",
    expected_output="Tweet",
    agent=social_media_manager
)

In [16]:
task4 = Task(
    description="""To meticulously review and harmonize the final output from both the blog manager and social media manager, ensuring cohesion and excellence in the final publication. Once done, publish the final report.""",
    expected_output="Final Report",
    agent=content_marketing_manager
)

## Define the Crew of Agents
* Crew with sequential process

In [22]:
agents = [online_researcher, blog_manager, social_media_manager, content_marketing_manager]

crew = Crew(
    agents=agents,
    tasks=[task1, task2, task3, task4],
    verbose=2
)

result = crew.kickoff()

print(result)

2024-12-04 02:22:43,708 - 8355854400 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Online Researcher
 [INFO]: == Starting Task: Write me a report on Agentic Behavior. After the research on Agentic Behavior,pass the 
    findings to the blog manager to generate the final blog article. Once done, pass it to the social media 
    manager to write a tweet on the subject.


> Entering new CrewAgentExecutor chain...
Thought: I need to gather detailed and reliable information on Agentic Behavior to create a comprehensive report. The first step is to use the tavily_search_results_json search engine to find current and relevant articles, studies, or reports on the topic.

Action: tavily_search_results_json
Action Input: {"query": "Agentic Behavior detailed report"} 

[{'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC10261990/', 'content': 'AGENTIC BEHAVIOR: Valenced agentic emotion (anger, exuberance) should motivate action and voluntary approach of risky rewards ... agentic circuit reactivity assessed by AMP-facilitated BOLD). We report trends at

## Before running the app
* Be careful, because running Multi-Agent apps like this with chatGPT-4 is going to be more expensive than running traditional apps. Pay attention to the total cost in LangSmith. For us it was around $0.21

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-basic-multiagent-crewai.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-basic-multiagent-crewai.py

## One important caveat: Multi-Agent Networks take time to deliver the final solution, so you will have to be patient.
* Multi-Agent Networks, also called LLM Apps with Agentic Behavior or Multi-Agent LLM Apps take time, but their results are often better than the ones got from Non-Agentic LLM Apps.

## You can then see workflow steps displayed in the terminal

#### See this amazing output we got from the online search agent in the first stage
* "Thought: Since the initial and subsequent search attempts have led to the same inaccessible ScienceDirect source, and given the constraints of my tools and the lack of alternative sources provided by the search tool, **I need to adapt my approach to fulfill the task without direct access to the content of the source**. I will synthesize a general understanding of agentic behavior based on the brief information available from the search result and general knowledge of the topic."
* Did you see? That was an AI Agent reasoning and making decisions in real time! Impressive. Imagine what the Multi-Agent LLM Apps will be able to do very soon!

#### The results of the app will improve by fine-tuning the backstories of the agents and the task descriptions.
* We leave this task up to you. Experiment and compare the ouputs you get with different approaches.

## Take a look at LangSmith
* See that the project takes a lot of tokens.
* See that each agent is monitored.
* Click on each agent and see input and output.